ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **24Sep2018**

# 05. Computational Linear Algebra Fundamentals

$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
$

---
## Table of Contents
* [Theory](#theory)
* [Matrix-vector product operations](#product)
* [NumPy and SciPy Linear Algebra](#linalg)
 + [Matrix solve](#solve)
 + [LU factorization](#lu)
---

## Theory<a id="theory"></a>
The notes found [here](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/ErfDAD_jL1tHkDrx29w89-MBcOgK4JAnEYSheRoxkL_0sw?e=fpzG5K) cover basic elements of lineary system of algebraic equations. Their representation in row and column formats and the insight gained from them when searching for a solution of the system.

Some basic theoretical aspects of solving for $\xvec$ in $\Amtrx\,\xvec = \bvec$ are covered. $\Amtrx$ is a matrix, $\bvec$ and $\xvec$ are vectors.

## Matrix-vector product operations<a id="product"></a>
The following operations between vectors and matrices are obtained directly from the buil-in functions in the `numpy` package.

In [41]:
'''Import the NumPy package as usual'''

import numpy as np

In [42]:
'''Vector scalar product or dot product of vectors'''

a_vec = np.array( np.random.random(3) )
b_vec = np.array( np.random.random(3) )

a_vec_dot_b_vec = np.dot( a_vec, b_vec ) # clear linear algebra operation
print('a.b =', a_vec_dot_b_vec)

a_vec_x_b_vec = a_vec @ b_vec   # linear algebra multiplication
print('a@b =', a_vec_x_b_vec )

a.b = 1.36116101113
a@b = 1.36116101113


In [43]:
'''Matrix-vector product'''

a_mtrx = np.array( [ [ 2.,  1., 1.],
                     [ 4., -6., 0.],
                     [-2.,  7., 2.]])

b_vec = np.array( [5., -2., 9.])

a_mtrx_x_b_vec = a_mtrx @ b_vec # linear algebra matrix-vector product

print('A x b =', a_mtrx_x_b_vec)

A x b = [ 17.  32.  -6.]


## NumPy and SciPy Linear Algebra<a id="linalg"></a>
NumPy has extensive support for [linear algebra](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html?highlight=linear%20algebra) arrays. We collect here the relevant operations for this course.
However additional resources are instead added to [SciPy](https://docs.scipy.org/doc/scipy-1.1.0/reference/) for general scientific computing including linear algebra.

Linear algebra operations are obtained from the `linalg` sub-package of the `numpy` package.

In [44]:
'''Import the NumPy package as usual'''

import numpy.linalg as linalg
from numpy import linalg

In [45]:
'''Vector norm (or magnitude)'''

norm_b_vec = linalg.norm( b_vec ) # default norm is the 2-norm 
print('||b|| =', norm_b_vec)      # same as magnitude

||b|| = 10.4880884817


*Solve*<a id="solve"></a> for $\xvec$ in $\Amtrx\,\xvec = \bvec$, where $\Amtrx = 
\begin{pmatrix}
2 & 1 & 1 \\
4 & -6 & 0 \\
-2 & 7 & 2
\end{pmatrix}
$
and $\bvec = \begin{pmatrix} 5\\ -2\\ 9 \end{pmatrix}$

In [46]:
'''Matrix solver (this is short for solution of a linear algebraic system of equations)'''

x_vec = linalg.solve( a_mtrx, b_vec ) # solve linear system for A, b
print('solution x =', x_vec)

solution x = [ 1.  1.  2.]


The residual vector, $\rvec = \bvec - \Amtrx\,\xvec$ is of importance. So is its norm $\norm{\rvec}$.

In [47]:
'''Verify the accuracy of the solution'''

res_vec = b_vec - a_mtrx @ x_vec
print('b - A x =',res_vec)
print('||b - A x|| =',linalg.norm( res_vec ))

b - A x = [ 0.  0.  0.]
||b - A x|| = 0.0


In [48]:
'''Matrix rank'''

k = linalg.matrix_rank( a_mtrx )
print('rank(A) =',k)
print('shape(A) =',a_mtrx.shape)
if k == a_mtrx.shape[0] and k == a_mtrx.shape[1]:     # flow control
    print('A is non-singular; solution is unique ')

rank(A) = 3
shape(A) = (3, 3)
A is non-singular; solution is unique 


In [49]:
'''Matrix determinant'''

det_a_mtrx = np.linalg.det( a_mtrx )
print('det(A) =', det_a_mtrx)

det(A) = -16.0


The inverse matrix is denoted as $\Amtrx^{-1}$ and is computed as the matrix that multiplies $\bvec$ and produces the solution $\xvec$.

In [50]:
'''Matrix inverse'''

a_mtrx_inv = np.linalg.inv( a_mtrx )
print('A^-1 =\n', a_mtrx_inv)

A^-1 =
 [[ 0.75   -0.3125 -0.375 ]
 [ 0.5    -0.375  -0.25  ]
 [-1.      1.      1.    ]]


Using the inverse, the same solution will be found: $\xvec = \Amtrx^{-1}\,\bvec$.

In [51]:
'''Solution using the inverse'''

x_vec_again = a_mtrx_inv @ b_vec  # matrix-vector multiply
print('solution x =', x_vec_again)

solution x = [ 1.  1.  2.]


This is the reciprocal of the matrix element-by-element $(\Amtrx)^{-1}$, which is very different than the inverse.

In [52]:
'''Inverse power of a matrix'''

#a_mtrx_to_negative_1 = a_mtrx**(-1) # this will cause an error (division by zero)

'Inverse power of a matrix'

### $\Pmtrx\Lmtrx\Umtrx$ <a id="lu"></a> factorization
The factors: $\Pmtrx$, $\Lmtrx$, and $\Umtrx$ where $\Pmtrx\Lmtrx\Umtrx = \Amtrx$ can be obtained from the SciPy linear algebra package:

In [53]:
'''L factor of A'''

from scipy import linalg

(p_mtrx, l_mtrx, u_mtrx) = linalg.lu(a_mtrx)
print('P =\n',p_mtrx)
print('L =\n',l_mtrx)
print('U =\n',u_mtrx)
print('PLU - A =\n',p_mtrx@l_mtrx@u_mtrx-a_mtrx)

P =
 [[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
L =
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  1.   1. ]]
U =
 [[ 4. -6.  0.]
 [ 0.  4.  1.]
 [ 0.  0.  1.]]
PLU - A =
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [55]:
'''PLU x = b'''

y_vec = linalg.solve(l_mtrx, p_mtrx.transpose() @ b_vec) # L y = PT b
x_vec = linalg.solve(u_mtrx, y_vec)                      # U x = y
x_vec - x_vec_again

array([ 0.,  0.,  0.])